# Download the library

In [3]:
# !pip install docplex
# !pip install cplex

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [4]:
from docplex.cp.model import *
import numpy as np

# Info

The format of these data files is:
number of planes (p), freeze time
for each plane i (i=1,...,p):
   - appearance time, 
   - earliest landing time, 
   - target landing time,
   - latest landing time, 
   - penalty cost per unit of time for landing before target, 
   - penalty cost per unit of time for landing after target

for each plane j (j=1,...p): separation time required after 
                                i lands before j can land


### Sigle runway definitions and variables

![image.png](Images/single_run.png)


### Multirunway defintions and variables 

![image.png](Images/multi_run.png)

## Get data

In [41]:
def read_datafiles(file):
    with open(file, 'r') as data:
        data_lines = data.readlines()
        number_planes = int(data_lines[0].split()[0])
        freeze_time =  int(data_lines[0].split()[1])
        mixed_data = [line.split() for line in data_lines[1:] if not line.isspace()]
        mixed_data = [[(float(j)) for j in i] for i in mixed_data]
        
        flight_details = np.empty([0,6],dtype=float)
        separation_time = np.empty([0,number_planes],dtype=float)
        
        flag = 0
       
        for element in mixed_data:
            if flag == 0: # flight details
                flight_details = np.vstack([flight_details, np.array(element)])
                flag = 1
                element_final = []
            else:  # separation_times
                element_final.extend(element)
                if len(element_final) == number_planes:
                    separation_time = np.vstack([separation_time, np.array(element_final)])
                    flag = 0
                
        print(f" number planes: {number_planes}")
        print(f" freeze time: {freeze_time}")
        print(f" mixed data: {mixed_data}")
        print(f" flight details: {flight_details}")
        print(f" separation time: {separation_time}")
    
    return number_planes, freeze_time, flight_details, separation_time


In [5]:
read_datafiles("./airland1.txt")

 number planes: 10
 freeze time: 10
 mixed data: [[54, 129, 155, 559, 10, 10], [99999, 3, 15, 15, 15, 15, 15, 15], [15, 15], [120, 195, 258, 744, 10, 10], [3, 99999, 15, 15, 15, 15, 15, 15], [15, 15], [14, 89, 98, 510, 30, 30], [15, 15, 99999, 8, 8, 8, 8, 8], [8, 8], [21, 96, 106, 521, 30, 30], [15, 15, 8, 99999, 8, 8, 8, 8], [8, 8], [35, 110, 123, 555, 30, 30], [15, 15, 8, 8, 99999, 8, 8, 8], [8, 8], [45, 120, 135, 576, 30, 30], [15, 15, 8, 8, 8, 99999, 8, 8], [8, 8], [49, 124, 138, 577, 30, 30], [15, 15, 8, 8, 8, 8, 99999, 8], [8, 8], [51, 126, 140, 573, 30, 30], [15, 15, 8, 8, 8, 8, 8, 99999], [8, 8], [60, 135, 150, 591, 30, 30], [15, 15, 8, 8, 8, 8, 8, 8], [99999, 8], [85, 160, 180, 657, 30, 30], [15, 15, 8, 8, 8, 8, 8, 8], [8, 99999]]
 flight details: [[ 54. 129. 155. 559.  10.  10.]
 [120. 195. 258. 744.  10.  10.]
 [ 14.  89.  98. 510.  30.  30.]
 [ 21.  96. 106. 521.  30.  30.]
 [ 35. 110. 123. 555.  30.  30.]
 [ 45. 120. 135. 576.  30.  30.]
 [ 49. 124. 138. 577.  30.  30.]
 [

(10, 10, array([[ 54., 129., 155., 559.,  10.,  10.],
        [120., 195., 258., 744.,  10.,  10.],
        [ 14.,  89.,  98., 510.,  30.,  30.],
        [ 21.,  96., 106., 521.,  30.,  30.],
        [ 35., 110., 123., 555.,  30.,  30.],
        [ 45., 120., 135., 576.,  30.,  30.],
        [ 49., 124., 138., 577.,  30.,  30.],
        [ 51., 126., 140., 573.,  30.,  30.],
        [ 60., 135., 150., 591.,  30.,  30.],
        [ 85., 160., 180., 657.,  30.,  30.]]), array([[9.9999e+04, 3.0000e+00, 1.5000e+01, 1.5000e+01, 1.5000e+01,
         1.5000e+01, 1.5000e+01, 1.5000e+01, 1.5000e+01, 1.5000e+01],
        [3.0000e+00, 9.9999e+04, 1.5000e+01, 1.5000e+01, 1.5000e+01,
         1.5000e+01, 1.5000e+01, 1.5000e+01, 1.5000e+01, 1.5000e+01],
        [1.5000e+01, 1.5000e+01, 9.9999e+04, 8.0000e+00, 8.0000e+00,
         8.0000e+00, 8.0000e+00, 8.0000e+00, 8.0000e+00, 8.0000e+00],
        [1.5000e+01, 1.5000e+01, 8.0000e+00, 9.9999e+04, 8.0000e+00,
         8.0000e+00, 8.0000e+00, 8.0000e+00, 

# The MIP model

In [6]:
def MIP_model(file_name,R):
    from docplex.mp.model import Model
    mdl = Model("Scheduling Aircraft Landing - Multi Runaway Static Case")
    
    P, freeze_time, flight_details, separation_time = read_datafiles(file_name)

    #Creating a CPLEX model
    #model=Model("Aircraft Landing Schedule")
    
    # The first column relates to the actual appearence time of the plane so will not be taken into account for our decision variables

    E = flight_details[:,1]  #earliest landing time,
    T = flight_details[:,2]  #target landing time,
    L = flight_details[:,3]  #latest landing time,
    g = flight_details[:,4]  #penalty cost per unit of time for landing before target,
    h = flight_details[:,5]  #penalty cost per unit of time for landing after target
    range_LT = max(flight_details[:,3]) - min(flight_details[:,1]) #landing range time
    ij = [(i,j) for i in np.arange(P) for j in np.arange(P)]
    ir = [(i, r) for i in np.arange(P) for r in np.arange(R)]
    

    # Defining decision variables

    alpha   = mdl.continuous_var_dict(np.arange(P),lb=0,ub=mdl.infinity, name="alpha") # how soon a plane lands before target time
    beta    = mdl.continuous_var_dict(np.arange(P),lb=0,ub=mdl.infinity, name="beta") # how soon a plane lands after target time
    x       = mdl.continuous_var_dict(np.arange(P),lb=0,ub=mdl.infinity, name="x") # landing time for a plane
    sigma   = mdl.binary_var_dict(ij,lb=0,ub=1, name="sigma") # binary variable: 1 if plane i lands before plane j, 0 elsewhere
    y       = mdl.binary_var_dict(ir,lb=0,ub=1, name="y") # 1 if plane i lands on runaway  r , 0 otherwise
    z       = mdl.binary_var_dict(ij,lb=0,ub=1, name="z") # 1 if planes i and  j land on the same runaway, 0 otherwise



    # Adding constraints
    
    mdl.add_constraints(x[i]>=E[i-1] for i in np.arange(P)) # Const 1 - Landing time of plane i must be later than the earliest landing time
    mdl.add_constraints(x[i]<=L[i-1] for i in np.arange(P)) # Const 1 - Landing time of plane i must be earlier than the before latest landing time
    mdl.add_constraints(sigma[i,j]+sigma[j,i]==1 for i in np.arange(P) for j in np.arange(P) if j!=i)  # Const 2 - Plain i must land before plain j or plain j before plain i
    mdl.add_constraints(alpha[i]>=T[i]-x[i] for i in np.arange(P)) # Const 14 - How soon plane i lands before T[i] must be larger than T[i] - x[i]
    mdl.add_constraints(alpha[i]<=T[i]-E[i] for i in np.arange(P)) # Const 15 - How soon plane i lands before T[i] must be smaller than T[i] - E[i]
    mdl.add_constraints(alpha[i]>= 0 for i in np.arange(P)) # Const 15 - How soon plane i lands before T[i] must be at least zero
    mdl.add_constraints(beta[i]>=x[i]-T[i] for i in np.arange(P))  # Const 16 - How soon plane i lands after T[i] must be larger than x[i] - T[i]
    mdl.add_constraints(beta[i]<=L[i]-T[i] for i in np.arange(P))  # Const 17 - How soon plane i lands after T[i] must be smaller than L[i] - T[i]
    mdl.add_constraints(beta[i]>= 0 for i in np.arange(P))  # Const 17 - How soon plane i lands after T[i] must be at least zero
    mdl.add_constraints(x[i]==T[i]-alpha[i]+beta[i] for i in np.arange(P))  # Const 18 - Landing time is equal to target time minus arriving early or plus arriving late
   
    mdl.add_constraints(mdl.sum(y[i,r] for r in np.arange(R))==1 for i in np.arange(P))     # Const 28 - Plane i can only land in 1 runaway
    mdl.add_constraints(z[i,j]==z[j,i] for i in np.arange(P) for j in np.arange(P) if j>i)  # Const 29 - Symetry constraint: If plane i lands in the same runaway as plane j, plane j lands in the same runaway as plane i
    mdl.add_constraints(z[i,j]>=y[i,r]+y[j,r]-1 for r in np.arange(R) for j in np.arange(P) for i in np.arange(P) if j>i) # Const 30 - If there is any runaway r for which y[i,r]=y[j,r]=1 then z[i,j]=1. If z[i,j]=0 then the planes i and j cannot land on the same runaway 
    
    mdl.add_constraints(x[j]-x[i]>=separation_time[i,j]*z[j,i] - (sigma[j,i])*range_LT for i in np.arange(P) for j in np.arange(P) if j!=i)   # Const 12 - Separation time between plane i and plane j must be respected
    # Const 31
    # Const 33


    cost_function = mdl.sum(beta[i] * h[i] + alpha[i] * g[i] for i in np.arange(P))

    mdl.minimize(cost_function)
    
    mdl.print_information()

    msol = mdl.solve()
    assert msol is not None, "model can't solve"

    return mdl

In [7]:
file_name = "./airland1.txt"
R = 2

mdl_ = MIP_model(file_name, R)
mdl_.report()
mdl_.print_solution()

 number planes: 10
 freeze time: 10
 mixed data: [[54, 129, 155, 559, 10, 10], [99999, 3, 15, 15, 15, 15, 15, 15], [15, 15], [120, 195, 258, 744, 10, 10], [3, 99999, 15, 15, 15, 15, 15, 15], [15, 15], [14, 89, 98, 510, 30, 30], [15, 15, 99999, 8, 8, 8, 8, 8], [8, 8], [21, 96, 106, 521, 30, 30], [15, 15, 8, 99999, 8, 8, 8, 8], [8, 8], [35, 110, 123, 555, 30, 30], [15, 15, 8, 8, 99999, 8, 8, 8], [8, 8], [45, 120, 135, 576, 30, 30], [15, 15, 8, 8, 8, 99999, 8, 8], [8, 8], [49, 124, 138, 577, 30, 30], [15, 15, 8, 8, 8, 8, 99999, 8], [8, 8], [51, 126, 140, 573, 30, 30], [15, 15, 8, 8, 8, 8, 8, 99999], [8, 8], [60, 135, 150, 591, 30, 30], [15, 15, 8, 8, 8, 8, 8, 8], [99999, 8], [85, 160, 180, 657, 30, 30], [15, 15, 8, 8, 8, 8, 8, 8], [8, 99999]]
 flight details: [[ 54. 129. 155. 559.  10.  10.]
 [120. 195. 258. 744.  10.  10.]
 [ 14.  89.  98. 510.  30.  30.]
 [ 21.  96. 106. 521.  30.  30.]
 [ 35. 110. 123. 555.  30.  30.]
 [ 45. 120. 135. 576.  30.  30.]
 [ 49. 124. 138. 577.  30.  30.]
 [

In [8]:
!pip install ortools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 82.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.8/409.8 KB 44.6 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.19.6
    Uninstalling protobuf-3.19.6:
      Successfully uninstalled protobuf-3.19.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.9.2 requires protobuf<3.20,>=3.9.2, but you have protobuf 4.21.12 which is incompatible.
tensorflow-metadata 1.12.0 requires protobuf<4,>=3.13, but you have protobuf 4.21.12 which is incompatible.
tensorboard 2.9.1 requires protobuf<3.20,>=3.9.2, but you have protobuf 4.21.12 which is incompatible.


In [51]:
def read_datafiles_cp(file):
    with open(file, 'r') as data:
        data_lines = data.readlines()
        number_planes = int(data_lines[0].split()[0])
        freeze_time =  int(data_lines[0].split()[1])
        mixed_data = [line.split() for line in data_lines[1:] if not line.isspace()]
        mixed_data = [[(float(j)) for j in i] for i in mixed_data]
        
        flight_details = np.empty([0,6],dtype=float)
        separation_time = np.empty([0,number_planes],dtype=float)
        
        flag = 0
       
        for element in mixed_data:
            if flag == 0: # flight details
                flight_details = np.vstack([flight_details, np.array(element)])
                flag = 1
                element_final = []
            else:  # separation_times
                element_final.extend(element)
                if len(element_final) == number_planes:
                    separation_time = np.vstack([separation_time, np.array(element_final)])
                    flag = 0

        minimum = min(flight_details[:,1])
        maximum = max(flight_details[:,3])
        num_periods = int(minimum % 10)
        int_size = maximum//num_periods
        period_constraints = []

        x = (0,0)
        lst = list(x)
        for i in range(num_periods):
          lst[0], lst[1] = int_size * i, int_size * (i+1)
          x = tuple(lst)
          period_constraints.append(x)

                
        print(f" number planes: {number_planes}")
        print(f" num_periods: {num_periods}")
        print(f" flight details: {flight_details}")
        print(f" period_constraints: {period_constraints}")
    
    return number_planes, num_periods, flight_details, period_constraints

In [52]:
read_datafiles_cp("./airland1.txt")

 number planes: 10
 num_periods: 9
 flight details: [[ 54. 129. 155. 559.  10.  10.]
 [120. 195. 258. 744.  10.  10.]
 [ 14.  89.  98. 510.  30.  30.]
 [ 21.  96. 106. 521.  30.  30.]
 [ 35. 110. 123. 555.  30.  30.]
 [ 45. 120. 135. 576.  30.  30.]
 [ 49. 124. 138. 577.  30.  30.]
 [ 51. 126. 140. 573.  30.  30.]
 [ 60. 135. 150. 591.  30.  30.]
 [ 85. 160. 180. 657.  30.  30.]]
 period_constraints: [(0.0, 82.0), (82.0, 164.0), (164.0, 246.0), (246.0, 328.0), (328.0, 410.0), (410.0, 492.0), (492.0, 574.0), (574.0, 656.0), (656.0, 738.0)]


(10, 9, array([[ 54., 129., 155., 559.,  10.,  10.],
        [120., 195., 258., 744.,  10.,  10.],
        [ 14.,  89.,  98., 510.,  30.,  30.],
        [ 21.,  96., 106., 521.,  30.,  30.],
        [ 35., 110., 123., 555.,  30.,  30.],
        [ 45., 120., 135., 576.,  30.,  30.],
        [ 49., 124., 138., 577.,  30.,  30.],
        [ 51., 126., 140., 573.,  30.,  30.],
        [ 60., 135., 150., 591.,  30.,  30.],
        [ 85., 160., 180., 657.,  30.,  30.]]), [(0.0, 82.0),
  (82.0, 164.0),
  (164.0, 246.0),
  (246.0, 328.0),
  (328.0, 410.0),
  (410.0, 492.0),
  (492.0, 574.0),
  (574.0, 656.0),
  (656.0, 738.0)])

In [57]:
def CP_model(file_name,R):
  P, freeze_time, flight_details, separation_time = read_datafiles(file_name)
  # Create variables
  alpha = {}
  beta = {}
  x = {}
  sigma = {}
  y = {}
  z = {}

  E = flight_details[:,1]  #earliest landing time,
  T = flight_details[:,2]  #target landing time,
  L = flight_details[:,3]  #latest landing time,
  g = flight_details[:,4]  #penalty cost per unit of time for landing before target,
  h = flight_details[:,5]  #penalty cost per unit of time for landing after target
  range_LT = max(flight_details[:,3]) - min(flight_details[:,1]) #landing range time
  ij = [(i,j) for i in np.arange(P) for j in np.arange(P)]
  ir = [(i, r) for i in np.arange(P) for r in np.arange(R)]

  from docplex.mp.model import Model # ELE AQUI TINHA docplex.mp.model, troquei para cp porque acho que é este o modulo para CP

  # Create the model
  mdl = Model("aircraft_landing_problem")

  # Read in the data
  num_aircrafts, num_periods, aircraft_data, period_constraints = read_datafiles_cp(file_name)

  # Create variables
  x = mdl.binary_var_matrix(num_aircrafts, num_periods, name="x") # 1 if aircraft i lands in period j, 0 otherwise
  s = mdl.continuous_var_matrix(num_aircrafts, num_periods, name="s") # start time for aircraft i in period j
  f = mdl.continuous_var_matrix(num_aircrafts, num_periods, name="f") # finish time for aircraft i in period j

  # Const 1 - An aircraft must land in exactly one period
  for i in range(num_aircrafts):
      mdl.add_constraint(mdl.sum(x[i, j] for j in range(num_periods)) == 1)

  # Const 2 - If aircraft i lands in period j, then s[i,j] and f[i,j] must be within the bounds for that period
  for i in range(num_aircrafts):
      for j in range(num_periods):
          mdl.add_constraint(s[i, j] >= period_constraints[j][0])
          mdl.add_constraint(f[i, j] <= period_constraints[j][1])
          mdl.add_constraint(s[i, j] <= f[i, j])
          mdl.add_constraint(f[i, j] - s[i, j] <= aircraft_data[i][3])
          mdl.add_constraint(s[i, j] - aircraft_data[i][1] >= 0)
          mdl.add_constraint(aircraft_data[i][2] - f[i, j] >= 0)

  # Const 3 - If aircraft i does not land in period j, then s[i,j] and f[i,j] must be zero
  for i in range(num_aircrafts):
      for j in range(num_periods):
          mdl.add_constraint(s[i, j] == 0).iff(x[i, j] == 0)
          mdl.add_constraint(f[i, j] == 0).iff(x[i, j] == 0)

  # Const 4 - The finish time for aircraft i in period j must be equal to the start time plus the required landing time
  for i in range(num_aircrafts):
      for j in range(num_periods):
          mdl.add_constraint(f[i, j] == s[i, j] + aircraft_data[i][4])

  # Const 5 - If aircraft i lands before aircraft j in period k, then s[i,k] <= f[j,k]
  for i in range(num_aircrafts):
      for j in range(num_aircrafts):
          if i != j:
              for k in range(num_periods):
                  mdl.add_constraint(s[i, k] <= f[j, k]).iff(x[i, k] + x[j, k] == 2)

  # Const 6 - If aircraft i lands in the same period as aircraft j, then there must be a minimum separation time between them
  for i in range(num_aircrafts):
      for j in range(num_aircrafts):
          if i != j:
              for k in range(num_periods):
                  mdl.add_constraint(f[i, k] - s[j, k] >= aircraft_data[i][5])

  # Define the objective function
  mdl.minimize(mdl.sum(x[i, j] * aircraft_data[i][6] for i in range(num_aircrafts) for j in range(num_periods)))

  # Solve the model
  mdl.solve()

  # Print the solution
  for i in range(num_aircrafts):
      for j in range(num_periods):
          if x[i, j].solution_value > 0:
              print("Aircraft {} lands in period {} at start time {} and finish time {}".format(i, j, s[i, j].solution_value, f[i, j].solution_value))





In [58]:
file_name = "./airland1.txt"
R = 2
CP_model(file_name, R)

 number planes: 10
 freeze time: 10
 mixed data: [[54, 129, 155, 559, 10, 10], [99999, 3, 15, 15, 15, 15, 15, 15], [15, 15], [120, 195, 258, 744, 10, 10], [3, 99999, 15, 15, 15, 15, 15, 15], [15, 15], [14, 89, 98, 510, 30, 30], [15, 15, 99999, 8, 8, 8, 8, 8], [8, 8], [21, 96, 106, 521, 30, 30], [15, 15, 8, 99999, 8, 8, 8, 8], [8, 8], [35, 110, 123, 555, 30, 30], [15, 15, 8, 8, 99999, 8, 8, 8], [8, 8], [45, 120, 135, 576, 30, 30], [15, 15, 8, 8, 8, 99999, 8, 8], [8, 8], [49, 124, 138, 577, 30, 30], [15, 15, 8, 8, 8, 8, 99999, 8], [8, 8], [51, 126, 140, 573, 30, 30], [15, 15, 8, 8, 8, 8, 8, 99999], [8, 8], [60, 135, 150, 591, 30, 30], [15, 15, 8, 8, 8, 8, 8, 8], [99999, 8], [85, 160, 180, 657, 30, 30], [15, 15, 8, 8, 8, 8, 8, 8], [8, 99999]]
 flight details: [[ 54. 129. 155. 559.  10.  10.]
 [120. 195. 258. 744.  10.  10.]
 [ 14.  89.  98. 510.  30.  30.]
 [ 21.  96. 106. 521.  30.  30.]
 [ 35. 110. 123. 555.  30.  30.]
 [ 45. 120. 135. 576.  30.  30.]
 [ 49. 124. 138. 577.  30.  30.]
 [

AttributeError: ignored

## Define the decision variables

## Define constraints

## Define the objective

## Solve with decision optimization